# NLC クロスバリデーションテスト#1

In [ ]:
!pip install --upgrade "watson-developer-cloud>=1.3.3"

In [ ]:
# The code was removed by DSX for sharing.

## ICOS(IBM Cloud Object Storage)からファイルを読み込む 

In [ ]:
# The code was removed by DSX for sharing.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, T_train, T_test = train_test_split(df_data["サンプル文"], df_data["正解クラス"], test_size=0.2)
# X_train 訓練データの入力
# X_test  テストデータの入力
# T_train 訓練データの正解ラベル
# T_test  テストデータの正解ラベル

print('train size:', X_train.shape[0])
print('test size:', X_test.shape[0])

In [ ]:
df_train = pd.concat([X_train,T_train],axis=1) #axis=1は横方向の結合を指定します。
df_train.head()

### トレーニング用データをCSVファイルに保存する

In [ ]:
DATA_SET = 'train.csv'
!rm -f $DATA_SET
df_train.to_csv(DATA_SET, sep=",", encoding="utf8", index=False, header=False)
!ls -l $DATA_SET
!head $DATA_SET

In [ ]:
df_test = pd.concat([X_test,T_test],axis=1) #axis=1は横方向の結合を指定します。
df_test.head()

In [ ]:
# The code was removed by DSX for sharing.

In [ ]:
from watson_developer_cloud import NaturalLanguageClassifierV1

natural_language_classifier = NaturalLanguageClassifierV1(
    username = credentials_nlc['username'],
    password = credentials_nlc['password'])

In [ ]:
natural_language_classifier.set_default_headers({'x-watson-learning-opt-out': "true"})

In [ ]:
def create_classifier(mode):
    # fetch all classifiers associated with the NLC instance
    result = natural_language_classifier.list_classifiers()
    # for the purposes of this demo, we handle only one classifier
    # return the first one found
    if mode == "list":
        if len(result['classifiers']) > 0:
            return result['classifiers']
        else:
            return {"error": "クラスを作成してください"}
    if mode == "add":
        print ('creating class ...')
        # if none found, create a new classifier, change this value
        with open(DATA_SET, 'rb') as training_data:
            metadata = '{"name": "abs_conc_class", "language": "ja"}'
            classifier = natural_language_classifier.create_classifier(
                metadata=metadata,
                training_data=training_data
            )
        return classifier

In [ ]:
create_classifier('list')

### クラスの作成

In [ ]:
myclass = create_classifier('add')
myclass

### クラスが使える状態かのチェック

In [ ]:
from time import sleep
from datetime import datetime, timezone, timedelta

JST = timezone(timedelta(hours=+9), 'JST')
#CID='e998c1x363-nlc-976' #上記 classifier_idをセットする
CID=myclass["classifier_id"]

while True:
    status = natural_language_classifier.get_classifier(CID)
    if status["status"] == "Available":
        print ('できたよ!!! 次に進め ...')
        break
    else:
        print ('まだだよ... ', datetime.now(JST).strftime("%Y/%m/%d %H:%M:%S"))
        sleep(60)

### テストデータでクラスを評価する

In [ ]:
#テストデータを使ってクラスを評価する
ok=0
ng=0
#for column_name, item in df_test.sample(30).iterrows():
#for column_name, item in df_test.head().iterrows():
for column_name, item in df_test.iterrows():
    #target = item[0][0:13] #先頭から15文字分取得
    target = item[0] # item[0]:1列目(サンプル文)を取得する
    classes = natural_language_classifier.classify(CID, target)
    if classes["classes"][0]["class_name"][:3] == item[1][:3]:
        ok += 1
    else:
        ng += 1
    print('[%5s] %4s(%0.2f)' % (classes["classes"][0]["class_name"][:3] == item[1][:3], # item[1]:2列目の最初の3文字 (教師データ)
                                classes["classes"][0]["class_name"], 
                                classes["classes"][0]["confidence"]), end="")
    
    print(' << (%s) %s >>' % (item[1], target))

print ('トータル = %d, OK = %d, NG = %d' % ((ok+ng),ok,ng))
print ('正解率 = %0.2f' % (ok/(ok+ng)))
print ('エラー率 = %0.2f' % (ng/(ok+ng)))

# (必要な場合) クラスの削除

In [ ]:
create_classifier('list')

In [ ]:
DELCID='e998c1x363-nlc-976' #上記 classifier_idをセットする
natural_language_classifier.delete_classifier(DELCID)

# バックアップ

In [ ]:
#これもうまくいったが...

for column_name, item in df_data.iloc[:,0].sample(10).iteritems():
    target = item[0:15] #先頭から15文字分取得
    classes = natural_language_classifier.classify(CLASS, target)
    print('%4s(%f)' % (classes["classes"][0]["class_name"], classes["classes"][0]["confidence"]), end="")
    print(' << %s >>' % target)

In [ ]:
#トレーニング用データフレームの作成
df_train = pd.DataFrame(X_train)
df_train['回答クラス'] = T_train
df_train.head()

In [ ]:
#テスト用データフレームの作成
df_test = pd.DataFrame(X_test)
df_test['回答クラス'] = T_test
df_test.head()

# データフレーム操作

In [ ]:
df_data.index

In [ ]:
df_data.columns

In [ ]:
df_data.shape

In [ ]:
df_data.info()

In [ ]:
df_data["サンプル文"].head()

In [ ]:
df_data.loc[:,"サンプル文"].head()

In [ ]:
df_data.iloc[:,0].head()